In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/depression_dataset/depression_dataset_reddit_cleaned.csv.zip")

In [ ]:
df.sample(5)

,clean_text,is_depression
1199,my doctor think that sertaline is better for anxiety but my fatigue specialist insists that citalopram is anyone with real life experience of either or both happy to share their opinion thanks,1
3065,must be depression fr,1
2303,this ha ruined my life the only thing keeping me from killing myself is my baby boy his mother manipulates me she s emotionally abusive and i don t know how to stand up to her i can t i m so afraid she ll take him away from me i can t keep doing this and i can t kill myself but i think i will i think he ll be better off without me i m a mess my finance are a mess my emotion i can barely take care of my own thing i m in so much turmoil i constantly fantasize about offing myself and every few day i sit there with a gun to my head and i just can t do it i think of my boy and then the pain is worse fyi we were engaged and shit fell apart for good last year october we still live together because we have a lease on a house and housing right now is a bitch nothing available so i m trapped here,1
763,i am no job no career suffering from severe depression and anxiety for year therapy med nothing help suffered emotionaly since teenage now m here all alone no one close totally done with life and thinking to end it thank u for reading,1
4704,endearingevania i sure wish i could go with you guy tmrw but a per usual school interferes,0


In [ ]:
df.shape

(7731, 2)

In [ ]:
!pip install -q -U torch
!pip install -q -U accelerate peft bitsandbytes trl
!pip install -q -U transformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 50.1 MB/s eta 0:00:00


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install transformers

In [ ]:
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
X_train = list()
X_test = list()
for label in [0,1]:
    train, test  = train_test_split(df[df.is_depression==label],
                                    train_size=800,
                                    test_size=200,
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)

X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)
X_train


,clean_text,is_depression
1006,hi all i m working in human resource for a smaller company and we re keen to making sure we re being inclusive we ve managed to implement flexibility for those who celebrate different religion belief new parent and a variety of physical disability but we re still learning when it come to neurodiversity generally we re very ambitious and strong believer in inclusion for all so we want to make sure we re going above and beyond rather than just doing the bear minimum my question for you is what process and or action have you seen company take that were groundbreaking or in some may made a major difference to you a someone with anxiety it can also be something you heard of rather than something that happened to you specifically if you don t know a company doing something that you feel would be very effective what would you love to see a company do,1
889,it still suck m that ha a job that a lot of people wish they had hr showed me that they received over 00 application for my position i moved to a new city for this job right before covid took over can t seem to make friend or meaningful connection with anyone in just over year i don t really have hobby and dread coming home to an empty house every night i ve always struggled with self esteem and this crushing loneliness ha me struggling yo get out of bed nobody call or even sends me text message i have no one that care about me i m already hiding so much including my emotion just desperately hoping someone i know reach out to me life is hard i get that and i know it s not easy for anyone but so many people i know have it so much easier their constant smile and laughter hurt but i hurt even more pretending to smile and laugh with them i ve ended up losing the passion and drive i once had got my career and am truly lost no friend no family no career no future fml,1
7709,did not sleep well at all and have a very unhappy stomach,0
691,doe someone feel the same i don t want to be alone with this feeling it s misery,1
4860,nicolerichie yes we had the vhs i cried when the old man died,0
...,...,...
2312,i hate myself i ve dreamed about finally gathering the courage to end my life for about year i have everything i need my basic need are met all paid by my caring family i m finishing a college degree also fully paid by my parent i feel weak and pathetic i see people with real issue traumatic shit who go on with their life while i m sitting here wanting to end my life without a reason why do i feel like this it feel invalid i ve never faced adversity every second of my life i ve spent tearing myself apart the only reason i haven t jumped in front of the metro is that i m scared to end up quadriplegic i m scared of everyone finding out that despite having my life served to me on a platter i still want to die i feel pathetic small and alone and i m scared i m starting to lose control of it before suicide felt like a refuge or a safe place to escape if necessary but now it s overwhelming and persistent i don t want to feel like this anymore but i can t stop it everything i do feel wrong i ve isolated myself from everyone and by the time i realized how lonely i felt i couldn t find the courage to reach out despite having everything given to me i feel like a failure and i hate everything about myself self loathing ha become the only way i interact with who i am and everything i do i feel like i don t deserve help i ve never faced adversity i m scared of failing to kill myself but i can t stop thinking about it,1
3570,lecturer are actually the most atrocious people who trigger a lot of depression rude when you ask for help setting quarter to impossible paper and then come to class and brag to student how they ve acquired their degree peak condescension,1
6251,just got back from picking up my parent goodbye house to myself,0
2416,i just want someone to give me a reason not to go down this spiral and no one will everyone ha their own life and he s going to leave me so

In [ ]:
eval_idx = [idx for idx in df.index if idx not in list(X_train.index) + list(X_test.index)]
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('is_depression', group_keys=False)
          .apply(lambda x: x.sample(n=150,random_state=10, replace=True)))
X_train = X_train.reset_index(drop=True)


In [ ]:
def generate_test_prompt(data_point):
    return f"""
            analyse the following sentence in square brackets and determine if it has depression or not, return the answer as 0 for no deression and 1 for depression.

            [{data_point["clean_text"]}] =
            """.strip()
y_true=X_test.is_depression
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["clean_text"])

def generate_prompt(data_point):
    return f"""
            analyse the following sentence in square brackets and determine if it has depression or not, return the answer as 0 for no deression and 1 for depression.

            [{data_point["clean_text"]}] = {data_point["is_depression"]}
            """.strip()

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1),
                       columns=["clean_text"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1),
                      columns=["clean_text"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)



In [ ]:
import pandas as pd

# Set the display option to show the full text in each cell
pd.set_option('display.max_colwidth', None)

# Now print the row data
row_index = 3
row_data = X_test.iloc[row_index]

print(f"Data for row {row_index}:\n{row_data}")

Data for row 3:
clean_text    analyse the following sentence in square brackets and determine if it has depression or not, return the answer as 0 for no deression and 1 for depression.\n\n            [fadyanwar it sad cause that wa the last gsm company owned by egyptian also orascom wa inforced to sell it share to fc] =
Name: 7018, dtype: object


In [ ]:
model_name = "microsoft/phi-2"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install transformers==4.37.0

  Using cached transformers-4.37.0-py3-none-any.whl (8.4 MB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [ ]:
import tqdm
def predict(test, model, tokenizer):
    y_pred = []
    for i in range(test.shape[0]):
        prompt = test.iloc[i]["clean_text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 2
                       )
        result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
        answer = result[0]['generated_text'].split("=")[-1].strip()
        if answer=='1':
            y_pred.append(1)
        elif answer=='0':
            y_pred.append(0)
        else: y_pred.append(-1)
    return y_pred

In [ ]:
import re

def get_num_layers(model):
    numbers = set()
    for name, _ in model.named_parameters():
        for number in re.findall(r'\d+', name):
            numbers.add(int(number))
    return max(numbers)

def get_last_layer_linears(model):
    names = []

    num_layers = get_num_layers(model)
    for name, module in model.named_modules():
        if str(num_layers) in name and not "encoder" in name:
            if isinstance(module, torch.nn.Linear):
                names.append(name)
    return names

In [ ]:
!pip install peft

In [ ]:
import os
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

In [ ]:

peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'dense'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


model = get_peft_model(model, peft_config)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    evaluation_strategy="epoch"
)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="clean_text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=512,
)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
trainer

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("trained-model")

Epoch,Training Loss,Validation Loss
1,1.364200,2.106042
2,1.271800,2.085766


In [ ]:
def evaluate(y_true, y_pred):


    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
prompt = X_test.iloc[0]["clean_text"]
pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 3,
                        temperature = 0.0,
                )
result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
answer = result[0]['generated_text'].split("=")[-1].strip()
answer

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

'0'

In [ ]:
y_pred=predict(X_test,model,tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [ ]:
y_true=list(y_true)
evaluate(y_true,y_pred)

Accuracy: 0.955
Accuracy for label 0: 0.985
Accuracy for label 1: 0.925

Classification Report:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.95      0.98      0.97       200
           1       0.98      0.93      0.95       200

    accuracy                           0.95       400
   macro avg       0.64      0.64      0.64       400
weighted avg       0.97      0.95      0.96       400


Confusion Matrix:
[[197   3]
 [ 11 185]]


In [ ]:
X_test.shape[0]

400